In [5]:
import csv

import jsonlines
import os
import pickle as pkl
from tqdm import tqdm
from collections import Counter
import csv

In [6]:
def get_mentions(set_name):
    data = []
    root_dir = set_name + "/"
    for file_name in os.listdir(root_dir):
        with open(root_dir+file_name, 'r') as f:
            reader = jsonlines.Reader(f)
            for x in reader:
                data.append(x)
    mentions = []
    for instance in data:
        sentences = []
        for item in instance['sentences']:
            sentences.extend(item)
        cluster = instance['clusters']
        for x in cluster:
            for y in x:
                mentions.append(" ".join(sentences[y[0]: y[1]+1]).strip().lower())
    with open("mentions/"+set_name+".pkl", 'wb') as f:
        pkl.dump(mentions, f)
    return mentions

In [7]:
def get_mention_clusters(set_name):
    data = []
    root_dir = "data/" + set_name + "/"
    for file_name in os.listdir(root_dir):
        with open(root_dir+file_name, 'r') as f:
            reader = jsonlines.Reader(f)
            for x in reader:
                data.append(x)
    mentions = []
    for instance in data:
        document = {}
        sentences = []
        for item in instance['sentences']:
            sentences.extend(item)
        document['sentences'] = sentences

        cluster = []
        for x in instance['clusters']:
            temp = []
            for y in x:
                temp.append(" ".join(sentences[y[0]: y[1]+1]).strip().lower())
            cluster.append(temp)
        document['clusters'] = cluster

        mentions.append(document)

    with open("mention_clusters/"+set_name+".pkl", 'wb') as f:
        pkl.dump(mentions, f)
    return mentions

In [4]:
datasets = ["arrau", "litbank", "ontonotes", "preco", "preco_fork", "qbcoref", "sara", "semeval"]
for dataset in tqdm(datasets):
    get_mention_clusters(dataset)

100%|██████████| 8/8 [00:40<00:00,  5.09s/it]


In [78]:
result = {}

root_dir = "mentions/"
for file_path in os.listdir(root_dir):
    file_name = file_path.strip().split('.')[0]
    with open(root_dir + file_path, 'rb') as f:
        mentions = pkl.load(f)
        mention_counter = Counter(mentions)
        temp = {}
        for token in ["this", "that", "there", "here", "then", "what", "who", "where", "when", "why"]:
            if token in mention_counter:
                temp[token] = mention_counter[token]
            else:
                temp[token] = 0
        temp["all"] = len(mentions)
        result[file_name] = temp

In [13]:
result = {}

root_dir = "mention_clusters/"
for file_path in os.listdir(root_dir):
    file_name = file_path.strip().split('.')[0]
    with open(root_dir + file_path, 'rb') as f:
        file_set = []
        mentions = pkl.load(f)
        for document in tqdm(mentions):
            temp = {}
            for item in ["this", "that", "there", "here", "then", "what", "who", "where", "when", "why"]:
                temp[item] = []
            clusters = document['clusters']
            for token in ["this", "that", "there", "here", "then", "what", "who", "where", "when", "why"]:
                for cluster in clusters:
                    if token in cluster:
                        temp[token].append(cluster)
            temp['sentences'] = document['sentences']
            file_set.append(temp)
        result[file_name] = file_set

with open('mention_clusters.pkl', 'wb') as f:
    pkl.dump(result, f)

100%|██████████| 1100/1100 [00:00<00:00, 12474.74it/s]


In [8]:
with open('mention_clusters.pkl', 'rb') as f:
    result = pkl.load(f)

In [9]:
output = {}
for item in result:
    temp = {}
    for token in ["this", "that", "there", "here", "then", "what", "who", "where", "when", "why"]:
        temp[token] = []

    for instance in result[item]:
        for token in ["this", "that", "there", "here", "then", "what", "who", "where", "when", "why"]:
            if token in instance:
                temp[token].extend(instance[token])

    output[item] = temp

In [10]:
print(output.keys())

dict_keys(['ontonotes', 'sara', 'preco_fork', 'qbcoref', 'preco', 'arrau', 'semeval', 'litbank'])


In [15]:
print(type(result['ontonotes']))
print(len(result['ontonotes']))
print(result['ontonotes'][1])

<class 'list'>
27944
{'this': [], 'that': [['this plot of the japanese army', 'that', 'a new strategy']], 'there': [], 'here': [], 'then': [], 'what': [], 'who': [], 'where': [], 'when': [], 'why': [], 'sentences': ['[CLS]', 'By', '1940', ',', 'China', "'", 's', 'War', 'of', 'Resistance', 'against', 'Japan', 'had', 'entered', 'a', 's', '##tale', '##mate', '.', 'The', 'situation', 'on', 'our', 'side', 'and', 'the', 'enemy', "'", 's', 'side', 'was', 'inter', '##twined', '.', 'The', 'Eighth', 'Route', 'Army', 'guerrilla', '##s', 'were', 'extra', '##ord', '##ina', '##rily', 'active', ',', 'creating', 'more', 'and', 'more', 'trouble', 'for', 'the', 'Japanese', 'army', 'in', 'North', 'China', '.', 'Hay', '##ao', 'Tad', '##a', ',', 'commander', 'of', 'the', 'Japanese', 'North', 'China', 'Area', 'Army', ',', 'adopted', 'a', 'strategy', 'of', 'siege', 'warfare', 'to', 'deal', 'with', 'the', 'Eighth', 'Route', 'Army', '.', 'The', 'specific', 'method', 'was', 'building', 'a', 'closely', 'connecte

In [11]:
for item in output['ontonotes']:
    print(item)
    for x in output['ontonotes'][item][:400]:
        print(x)
    print('=='*50)

this
['this section', 'this']
["shi ' na ##o mountain", "shi ' na ##o mountain", "des ##olate shi ' na ##o mountain", "towering and rugged shi ' na ##o mountain", 'the strategic passage', "shi ' na ##o mountain", "shi ' na ##o mountain", 'this']
['the meeting', 'met', 'this']
['focus today', 'this']
['it', 'this', 'his visit', 'his visit to ya ##su ##ku ##ni shrine', "the prime minister ' s visit to ya ##su ##ku ##ni shrine", 'it']
['selling', 'it', 'this']
['this road he takes', 'a very dangerous road', 'this', 'the wrong road']
['it', 'this issue', 'the issue', 'it', 'his visits to ya ##su ##ku ##ni shrine', 'this']
['prohibit ##ing', 'this']
['this', 'sent']
["200 billion us dollars ' worth of economic trade", 'this']
['the four northern islands issue dead knot', 'this', 'the four northern islands issue', 'the four northern islands issue', 'the northern territories issue']
['this', 'this']
['it', 'the talks', 'itself', 'this', 'this discussion', 'it']
['only 700 - plus million', 'th

In [23]:
check = ['this section', 'that']
print("this" in check)

False


In [88]:
result = []

root_dir = "mentions/"
for file_path in os.listdir(root_dir):
    file_name = file_path.strip().split('.')[0]
    with open(root_dir + file_path, 'rb') as f:
        mentions = pkl.load(f)
        mention_counter = Counter(mentions)
        temp = {}
        temp["dataset"] = file_name
        for token in ["this", "that", "there", "here", "then", "what", "who", "where", "when", "why"]:
            if token in mention_counter:
                temp[token] = mention_counter[token]
            else:
                temp[token] = 0
        temp["all"] = len(mentions)
        result.append(temp)

label = ["dataset", "this", "that", "there", "here", "then", "what", "who", "where", "when", "why", "all"]


try:
    with open('csv_dct.csv', 'w') as f:
        writer = csv.DictWriter(f, fieldnames=label)
        writer.writeheader()
        for elem in result:
            writer.writerow(elem)
except IOError:
    print("I/O error")

In [85]:
for x in result:
    print(x)

{'dataset': 'ontonotes', 'this': 11656, 'that': 16376, 'there': 96, 'here': 56, 'then': 96, 'what': 24, 'who': 24, 'where': 0, 'when': 8, 'why': 0, 'all': 1555816}
{'dataset': 'sara', 'this': 0, 'that': 0, 'there': 0, 'here': 0, 'then': 0, 'what': 0, 'who': 0, 'where': 0, 'when': 0, 'why': 0, 'all': 4011}
{'dataset': 'preco_fork', 'this': 5895, 'that': 4262, 'there': 138, 'here': 78, 'then': 6063, 'what': 12, 'who': 24, 'where': 0, 'when': 0, 'why': 49, 'all': 1503425}
{'dataset': 'qbcoref', 'this': 49, 'that': 14, 'there': 0, 'here': 7, 'then': 0, 'what': 0, 'who': 154, 'where': 0, 'when': 0, 'why': 0, 'all': 66297}
{'dataset': 'preco', 'this': 15407, 'that': 10922, 'there': 501, 'here': 263, 'then': 15961, 'what': 33, 'who': 103, 'where': 15, 'when': 30, 'why': 98, 'all': 4046337}
{'dataset': 'arrau', 'this': 109, 'that': 167, 'there': 245, 'here': 42, 'then': 9, 'what': 20, 'who': 153, 'where': 21, 'when': 13, 'why': 0, 'all': 72007}
{'dataset': 'semeval', 'this': 0, 'that': 0, 'the

In [80]:
print(result)

{'ontonotes': {'this': 11656, 'that': 16376, 'there': 96, 'here': 56, 'then': 96, 'what': 24, 'who': 24, 'where': 0, 'when': 8, 'why': 0, 'all': 1555816}, 'sara': {'this': 0, 'that': 0, 'there': 0, 'here': 0, 'then': 0, 'what': 0, 'who': 0, 'where': 0, 'when': 0, 'why': 0, 'all': 4011}, 'preco_fork': {'this': 5895, 'that': 4262, 'there': 138, 'here': 78, 'then': 6063, 'what': 12, 'who': 24, 'where': 0, 'when': 0, 'why': 49, 'all': 1503425}, 'qbcoref': {'this': 49, 'that': 14, 'there': 0, 'here': 7, 'then': 0, 'what': 0, 'who': 154, 'where': 0, 'when': 0, 'why': 0, 'all': 66297}, 'preco': {'this': 15407, 'that': 10922, 'there': 501, 'here': 263, 'then': 15961, 'what': 33, 'who': 103, 'where': 15, 'when': 30, 'why': 98, 'all': 4046337}, 'arrau': {'this': 109, 'that': 167, 'there': 245, 'here': 42, 'then': 9, 'what': 20, 'who': 153, 'where': 21, 'when': 13, 'why': 0, 'all': 72007}, 'semeval': {'this': 0, 'that': 0, 'there': 0, 'here': 0, 'then': 0, 'what': 0, 'who': 0, 'where': 0, 'when':